In [1]:
import numpy as np
import pandas as pd

In [6]:
with open('./data/tangshi.txt', 'r', encoding='utf-8') as f:
    data = f.readlines()

In [7]:
data=''.join(data)

In [8]:
len(data)

33004

In [9]:
len(set(data))

2624

In [10]:
data[1000:1200]

'言絕句\n詩文:(押虞韻)寒雨連江夜入吳，平明送客楚山孤。洛陽親友如相問，一片冰心在玉壺。\n\n詩名:閨怨\n作者:王昌齡\n詩體:七言絕句\n詩文:(押尤韻)閨中少婦不知愁，春日凝妝上翠樓。忽見陌頭楊柳色，悔教夫婿覓封侯。\n\n詩名:春宮曲\n作者:王昌齡\n詩體:七言絕句\n詩文:(押豪韻)昨夜風開露井桃，未央前殿月輪高。平陽歌舞新承寵，簾外春寒賜錦袍。\n\n詩名:長信怨\n作者:王昌齡\n詩體:樂府\n詩文:奉帚平明'

In [11]:
chars = list(set(data))

In [12]:
# data I/O
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print(f'data has {data_size} characters, {vocab_size} unique.')
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

data has 33004 characters, 2624 unique.


In [18]:
# hyperparameters
hidden_size = 256 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

In [19]:
hidden_size

256

In [20]:
# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

In [21]:
def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [22]:
def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

In [23]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print('----\n %s \n----' % (txt, ))

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0:
        print('iter %d, loss: %f' % (n, smooth_loss)) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 

----
 面耳妃觱建詣薛紛映嬋聆鼙欸遞尾尉寂追阻跼本患婀眠腳擣大娉謝百坤漏押鬟結燄差柯操賣柱標綠郎殲怕楓嗚颼摵柳妃二谿柔餞蝴洮姊烜了幕格處吮惡移抽平蹄滑實吾廚布腐鴛騎澒裘侵歛映馬邛酌摧嬌撚傳箭茲峰漱膽探吠生愁物沸怕梗攬消犁奠悠訖荷何鼙麒鄰撚慕黃蛾沒望廢虛渺突兮才篁壯著摧倒精惘絳嫌涓際匪聲藻潮都戚食螻岩惻橐猱濯賜琵酸觱馳藝亭嵩露宵翻藁嵋蛟隸夔扶千蕃縣淩騎空其妻吹芬蕃訓昭愚采支霹腰感裁藜轉戮終早賦喪摐渠鼉奉列泌 
----
iter 0, loss: 196.811375
----
 俗競雞櫳俗競雞犬俗競雞犬俗，雞櫳俗競不犬俗競雞犬俗處房犬俗競雞犬俗競爭黃年物競驚犬俗競雞櫳俗，雞犬俗，雞櫳俗競驚集俗競驚犬俗競驚集俗競驚犬俗競爭犬俗曾競房集雲，雞犬俗競雞犬俗競雞犬俗競雞犬俗競驚犬俗競雞集俗競雞犬俗競雞犬俗競爭犬俗曾。房櫳物競雞犬俗，雞櫳俗競雞集俗競雞犬俗，雞犬俗競驚犬俗酒爭犬俗競雞犬俗競爭在集俗競雞犬俗，驚犬俗，驚犬俗競房櫳俗競雞犬俗競雞犬俗競房犬俗競驚犬俗競爭在集物競雞犬俗競房 
----
iter 100, loss: 233.225817
----
 淚峨棧血紆峨棧血漫峨嵋血紆峨埃索漫峨棧索漫峨棧血紆峨嵋索紆峨嵋血淚峨棧血紆峨棧血漫峨棧索紆峨埃索紆峨棧血淚峨棧索漫峨嵋血漫峨棧索紆峨棧索漫峨棧索淚峨棧索淚峨埃索紆峨棧索漫峨棧索淚峨嵋索紆峨埃索紆峨嵋血淚峨埃血紆峨棧索漫峨埃血漫峨棧血漫峨棧血淚峨嵋血漫峨棧索紆峨埃索漫峨棧索淚峨棧索漫峨埃索紆峨埃血淚峨嵋血淚峨嵋血淚峨埃索淚峨埃血漫峨棧索淚峨嵋血紆峨埃血漫峨棧血淚峨棧血漫峨棧血紆峨埃血紆峨棧血漫峨嵋血 
----
iter 200, loss: 257.514332
----
 甘武判帶甘武判短甘武判，甘武判，甘場判人甘好判在甘武判官甘出判客甘武判，甘武判鄉甘武判苦甘武判音翁武判蘆甘武判在淒武判在甘武判聊:武判各甘武判宮羽武判，甘武判兵甘鄜判歲甘武判凝甘武判娘甘武判凝甘笑判在甘武判綠甘足判如甘武判苦聲武判琶甘武判乍甘武判在甘晴判在甘可判色甘武判，甘武判水甘武判急甘武判，甘武判城甘武判穴甘武判有甘武判短甘武判，甘語判外甘武判倚甘武判目甘平判堂甘武判在甘武判人甘武判，甘武判， 
----
iter 300, loss: 279.303686
----
 雌尺詩凋雌尺天凋雌尺天凋雌尺吹凋雌尺燕凋雌尺天凋雌尺否凋雌尺天凋雌尺鶯凋

KeyboardInterrupt: 

In [ ]:
all_vars = {}
all_vars['vocab_size'] = vocab_size
all_vars['Wxh'] = Wxh
all_vars['Whh'] = Whh
all_vars['bh'] = bh
all_vars['Why'] = Why
all_vars['by'] = by
all_vars['ix_to_char'] = ix_to_char
all_vars['char_to_ix'] = char_to_ix

all_vars['hprev'] = hprev
all_vars['inputs'] = inputs


import pickle

with open('rnn_chuci_20180528.pickle', 'wb') as handle:
    pickle.dump(all_vars, handle, protocol=pickle.HIGHEST_PROTOCOL)